In [58]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from bs4.element import Tag

In [3]:
with open("data/train_data.json", "r") as f:
    train_data = json.load(f)

In [5]:
len(train_data)

7683

In [168]:
html = train_data[100]["content"]["fullTextHtml"]

In [155]:
soup = BeautifulSoup(html, features="lxml")
text = soup.get_text()
lines = [line.strip() for line in text.splitlines()]
print("\n".join(lines))

Investigators said Tuesday that two sisters who killed their father after years of abuse should face murder charges in a case highlighting Russia's dire record on domestic violence.
Three Russian sisters -- Krestina, Angelina and Maria Khachaturyan -- stabbed their father Mikhail to death in July, 2018 after suffering years of beatings and sexual assault.
They were 17, 18 and 19 at the time.
Russia's Investigate Committee said in a statement on Tuesday that it had completed a probe into the killing and was recommending charges of murder and conspiracy to commit murder against the two older sisters, Krestina and Angelina.
The probe established that the sisters, acting as a group, stabbed their father with a knife and beat him with a hammer, causing fatal injuries.
It pointed to "mitigating circumstances" but said the two older sisters were of sound mind and aware of their actions at the time of the attack.
They face up to 20 years in prison if convicted. Investigators recommended that t

In [156]:
blocks = ["p", "h", "h1", "h2", "h3", "h4", "h5", "li", "u", "div"]

def _extract_blocks(parent_tag) -> list:
    extracted_blocks = []
    for tag in parent_tag:
        if tag.name in blocks:
            extracted_blocks.append(tag)
            continue
        if isinstance(tag, Tag):
            if len(tag.contents) > 0:
                inner_blocks = _extract_blocks(tag)
                if len(inner_blocks) > 0:
                    extracted_blocks.extend(inner_blocks)
    return extracted_blocks

def to_plaintext(html_text: str) -> str:
    soup = BeautifulSoup(html_text, features="lxml")
    extracted_blocks = _extract_blocks(soup.body)
    extracted_blocks_texts = [block.get_text().strip() for block in extracted_blocks]
    return "\n".join(extracted_blocks_texts)

In [159]:
print(to_plaintext(html))

Investigators said Tuesday that two sisters who killed their father after years of abuse should face murder charges in a case highlighting Russia's dire record on domestic violence.
Three Russian sisters -- Krestina, Angelina and Maria Khachaturyan -- stabbed their father Mikhail to death in July, 2018 after suffering years of beatings and sexual assault.
They were 17, 18 and 19 at the time.
Russia's Investigate Committee said in a statement on Tuesday that it had completed a probe into the killing and was recommending charges of murder and conspiracy to commit murder against the two older sisters, Krestina and Angelina.
The probe established that the sisters, acting as a group, stabbed their father with a knife and beat him with a hammer, causing fatal injuries.
It pointed to "mitigating circumstances" but said the two older sisters were of sound mind and aware of their actions at the time of the attack.
They face up to 20 years in prison if convicted. Investigators recommended that t

In [169]:
#load the processed data
with open("processed_data/processed_train_data.json", "r") as f:
    p_train_data = json.load(f)

In [173]:
print(p_train_data[100]["content"]["fullText"])

Investigators said Tuesday that two sisters who killed their father after years of abuse should face murder charges in a case highlighting Russia's dire record on domestic violence.
Three Russian sisters -- Krestina, Angelina and Maria Khachaturyan -- stabbed their father Mikhail to death in July, 2018 after suffering years of beatings and sexual assault.
They were 17, 18 and 19 at the time.
Russia's Investigate Committee said in a statement on Tuesday that it had completed a probe into the killing and was recommending charges of murder and conspiracy to commit murder against the two older sisters, Krestina and Angelina.
The probe established that the sisters, acting as a group, stabbed their father with a knife and beat him with a hammer, causing fatal injuries.
It pointed to "mitigating circumstances" but said the two older sisters were of sound mind and aware of their actions at the time of the attack.
They face up to 20 years in prison if convicted. Investigators recommended that t

In [49]:
# get all tags
all_tags = []
for e in train_data:
    html = e["content"]["fullTextHtml"]
    soup = BeautifulSoup(html, features="html.parser")
    all_tags += [tag.name for tag in soup.find_all()]

print(set(all_tags))

{'figure', 'tr', 'h3', 'u', 'strong', 'a', 'div', 'td', 'table', 'img', 'li', 'h', 'meta', 'sup', 'ul', 'span', 'em', 'h1', 'sub', 'figcaption', 'ol', 'h2', 'p'}


In [6]:
with open("data/taxonomy_mappings.json", "r") as f:
    taxonomy_mappings = json.load(f)

In [7]:
taxonomy_mappings

{'0': 'arts, culture and entertainment',
 '1': 'automotive',
 '2': 'crime, law and justice',
 '3': 'disaster and accident',
 '4': 'economy, business and finance',
 '5': 'education',
 '6': 'environment',
 '7': 'health',
 '8': 'labour',
 '9': 'lifestyle',
 '10': 'news',
 '11': 'non-standard content',
 '12': 'pets',
 '13': 'politics',
 '14': 'religion and belief',
 '15': 'science and technology',
 '16': 'society',
 '17': 'sport',
 '18': 'unrest, conflicts and war'}

In [11]:
with open("data/predict_payload.json", "r") as f:
    pred_payload = json.load(f)

In [25]:
content_df = pd.DataFrame(list(pd.DataFrame(train_data)['content']))

In [27]:
content_df["sections"].value_counts()

sections
[news -> world]                                        1720
[bi -> tech]                                           1218
[bi -> politics]                                        770
[bi -> finance]                                         624
[sports -> football]                                    565
[bi -> lifestyle]                                       419
[lifestyle -> mens-health]                              414
[bi -> strategy]                                        398
[news]                                                  326
[lifestyle -> womens-health]                            321
[bi -> sports]                                          248
[the-new-york-times -> world]                           236
[entertainment]                                         196
[the-new-york-times -> entertainment]                    82
[video]                                                  68
[news -> politics]                                       32
[lifestyle -> beauty-health]   